In [61]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from typing import List

# Project Description
This is a FFN trained to add integers from -10 to + 10

In [15]:
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

GPU: NVIDIA GeForce GTX 1660 Ti with Max-Q Design is available.


In [96]:
# create the dataset
N = 10000
torch.manual_seed(42)

nums_1 = torch.randint(-10, 11, (N,1), dtype=torch.float32)
nums_2 = torch.randint(-10, 11, (N,1), dtype=torch.float32)

target = nums_1 + nums_2

In [97]:
# Concatenate along columns (dim=1)
features = torch.cat((nums_1, nums_2), dim=1)

train_data,test_data, train_labels,test_labels = train_test_split(features, target, test_size=.2)

# then convert them into PyTorch Datasets (note: already converted to tensors)
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# finally, translate into dataloader objects
batchsize    = 16
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

In [114]:
test_data.tensors[0].shape[0]

2000

In [98]:
class addMachine(nn.Module):
  def __init__(self):
    super().__init__()

    ### input layer
    self.input = nn.Linear(2,16)

    ### hidden layers
    self.fc1 = nn.Linear(16,32)
    self.fc2 = nn.Linear(32,32)

    ### output layer
    self.output = nn.Linear(32,1)

  def forward(self,x):
    x = F.relu( self.input(x) )
    x = F.relu( self.fc1(x) )
    x = F.relu( self.fc2(x) )
    return self.output(x)

In [ ]:
#test model on some data from the train_loader
littleData = next(iter(train_loader))

model = addMachine()

print(littleData)
model(littleData[0])

[tensor([[ -8.,  -1.],
        [ -8.,  -3.],
        [  2.,  -1.],
        [  0.,   9.],
        [  2.,  -1.],
        [ -4.,  -4.],
        [  0.,   5.],
        [ -4.,  -6.],
        [  7.,   4.],
        [  7.,   8.],
        [  6.,  -8.],
        [  3.,  -7.],
        [  7.,  10.],
        [  2.,   5.],
        [  6.,  -1.],
        [-10.,   7.]]), tensor([[ -9.],
        [-11.],
        [  1.],
        [  9.],
        [  1.],
        [ -8.],
        [  5.],
        [-10.],
        [ 11.],
        [ 15.],
        [ -2.],
        [ -4.],
        [ 17.],
        [  7.],
        [  5.],
        [ -3.]])]


tensor([[-0.0527],
        [ 0.0109],
        [-0.1462],
        [-0.1250],
        [-0.1462],
        [-0.0765],
        [-0.1544],
        [-0.0379],
        [-0.0594],
        [-0.0813],
        [-0.1147],
        [-0.1323],
        [-0.0740],
        [-0.1420],
        [-0.0435],
        [-0.1801]], grad_fn=<AddmmBackward0>)

In [113]:
for X,y in train_loader:
    print(X,y)
    break

tensor([[-5.,  4.],
        [-2.,  3.],
        [ 0.,  1.],
        [-1., -6.],
        [-9., 10.],
        [10.,  9.],
        [ 8., -9.],
        [ 7.,  4.],
        [-9., -6.],
        [-6.,  4.],
        [-2.,  4.],
        [ 0.,  9.],
        [ 2., 10.],
        [ 2., -5.],
        [10., 10.],
        [10., -4.]]) tensor([[ -1.],
        [  1.],
        [  1.],
        [ -7.],
        [  1.],
        [ 19.],
        [ -1.],
        [ 11.],
        [-15.],
        [ -2.],
        [  2.],
        [  9.],
        [ 12.],
        [ -3.],
        [ 20.],
        [  6.]])


In [115]:
# train the model

# global parameter
numepochs = 1000

def trainTheModel(train_loader, test_loader, model):

  # loss function and optimizer
  lossfun = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=.001)

  #losses
  trainLoss = torch.zeros(numepochs)
  testLoss  = torch.zeros(numepochs)

  for epochi in range(numepochs):

    # switch on training mode
    model.train()

    # loop over training data batches
    batchLoss = []
    for X,y in train_loader:

      # forward pass and loss
      yHat = model(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())
    # end of batch loop...

    # and get average losses across the batches
    trainLoss[epochi] = np.mean(batchLoss)

    # test accuracy
    model.eval()
    X,y = next(iter(test_loader)) # extract X,y from test dataloader
    with torch.no_grad(): # deactivates autograd
      yHat = model(X)
    testLoss[epochi] = lossfun(yHat, y)

  # function output
  return trainLoss, testLoss

In [ ]:
addition_model = addMachine()